#Runnable PassThrough
Runnable pass-throughs are used to pass the value of a previous step in a chain to the next step without modification.
This is useful for scenarios where you need to use the output of one step as input for another step (not exactly next but steps following it).

In [38]:
!pip install -qU langchain langchain-groq langchain_community langchain-huggingface faiss-gpu

In [39]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [40]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [42]:
# Setup for vector store and retriever
vectorstore = FAISS.from_texts(
    ["Vinod kamli is oldest friend of Sachin"], embedding=embeddings
)

In [44]:
retriever = vectorstore.as_retriever()

# Define the prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatGroq()

# Create the chain with retrieval and pass-through
retrieval_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }  # Pass-through step
    | prompt
    | model
    | StrOutputParser()
)

In [46]:
# Example invocation
result = retrieval_chain.invoke("Who is the oldest friend of Sachin?")
print(result)

Based on the provided context, Vinod Kamli is the oldest friend of Sachin.
